In [1]:
import sys
sys.path.append("/home/profilum/profilum/")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import re
import pymorphy2
import random
import numpy as np
import nltk
from fn.monad import optionable
from tqdm import tqdm
from sqlalchemy.orm import *
from db import *
from joblib import delayed, Parallel
import gensim
import json
from datetime import datetime
import pandas as pd
from lxml.html import document_fromstring
from lxml.etree import tounicode
from utils import *
from currency_converter import CurrencyConverter, RateNotFoundError
from lib.hh_uskills import get_uskills
import pickle
nltk.data.path.append("/home/profilum/.nltk")

# Парсинг

In [4]:
CurrencyConverterApi = CurrencyConverter()

# Those currencies are not supported by api
not_supported = {
    'RUR': 1,
    'KZT': 0.180672506,
    'BYR': 30.33,
    'UAH': 2.15411133,
    'AZN': 33.59
}


def convert_price(cost_currency_date: list) -> int:
    '''Convert vacancy sallary'''

    # Get API of CurrencyConverter
    global CurrencyConverterApi, not_supported

    # Set cost, currency, date
    cost = cost_currency_date[0]
    currency = cost_currency_date[1]
    open_time = cost_currency_date[2]

In [5]:
all_raws = session.query(Raw).filter(Raw.http_errcode == None).all()

In [6]:
def parse(html: str, raw_id: int) -> dict:
    """Parse raw html into a Vacancy."""

    # FIXME: handle `html == ''`
    html = html.replace('<li>', '\n').replace('</li>', '\n').replace(
        '<p>', '\n').replace('</p>', '\n')

    # Parse html
    root = document_fromstring(html)

    # Get time of creating
    time = root.xpath("//time")

    if len(time) > 0:
        time = time[0].get('datetime')

        if time:
            # Parse time
            time = time.split("T")[0]
            time = datetime.strptime(time, "%Y-%m-%d")
    else:
        time = None

    # Try to get the price from meta
    try:
        # Get currency {'AZN', 'BYR', 'EUR', 'KZT', 'RUR', 'UAH', 'USD'}
        currency = root.xpath("//meta[contains(@itemprop,'salaryCurrency')]")[
            0].get('content')

        # Get price
        cost = int(
            root.xpath("//meta[contains(@itemprop,'baseSalary')]")[0].get(
                'content'))

        # Convert it
        rub_for_job = convert_price([cost, currency, time])

    except IndexError:
        # з/п не указана
        rub_for_job = 0

    # Find text
    text = root.xpath("//div[contains(@class,'b-vacancy-desc-wrapper')]")[0]
    # Got it!
    text = text.text_content()

    return {
        'open_time': time,
        'text': text,
        'salary': rub_for_job,
        'raw_id': raw_id
    }

In [7]:
new_parsed = Parallel(
    n_jobs=12, verbose=3)(delayed(parse)(x.html, x.id) for x in all_raws)

[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 15160 tasks      | elapsed:    2.3s
[Parallel(n_jobs=12)]: Done 53880 tasks      | elapsed:    7.3s
[Parallel(n_jobs=12)]: Done 108088 tasks      | elapsed:   14.5s
[Parallel(n_jobs=12)]: Done 177784 tasks      | elapsed:   23.5s
[Parallel(n_jobs=12)]: Done 262968 tasks      | elapsed:   34.9s
[Parallel(n_jobs=12)]: Done 363640 tasks      | elapsed:   48.2s
[Parallel(n_jobs=12)]: Done 479800 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 611448 tasks      | elapsed:  1.4min
[Parallel(n_jobs=12)]: Done 758584 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 921208 tasks      | elapsed:  2.0min
[Parallel(n_jobs=12)]: Done 1099320 tasks      | elapsed:  2.4min
[Parallel(n_jobs=12)]: Done 1292920 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 1502008 tasks      | elapsed:  3.3min
[Parallel(n_jobs=12)]: Done 1628453 tasks      | elapsed:  3.7min
[Parallel(n_jobs=12)]: Don

In [8]:
new_texts = {i['raw_id']: i['text'] for i in new_parsed}

In [9]:
all_vacancies = session.query(Vacancy).all()

In [10]:
for i in tqdm(range(len(all_vacancies))):
    all_vacancies[i].text = new_texts[all_vacancies[i].raw_id]

100%|██████████| 841447/841447 [00:03<00:00, 236943.48it/s]


# Достаём скиллы

In [11]:
@optionable
def cut_by_semmarkers(description: str):
    description = description.lower()
    
    a = ""
    for i in ['кандидатам:', 'требования:', 'обязанности:']:
        if i in description:
            a += description.split(i)[1].split(':')[0]

#         re.match(
#         r'(требования|обязанности):.*?:',
#         description,
#         flags=(re.IGNORECASE + re.DOTALL))
    return a

In [12]:
def split_skill(a):
    main_skill = a
    repl = lambda x: x.replace(',', '|').replace(
                    ' или ', '|').replace(' и ', '|').replace('/', '|').split('|')
    
    first_match = re.match(r'([а-яА-ЯA-Za-z\-\s]*([,и\/]|или)\s)', a)

    if first_match:
        first_match = first_match.group()
        a = repl(a.replace(first_match, ''))

        for i in a:
            if len(i.split()) > 4:
                return repl(main_skill)
        
        variants = first_match.replace(',', ' ').replace(
            ' или ', ' ').replace(' и ', ' ').replace('/', ' ')
        variants = list(filter(len, variants.split(' ')))
        if len(variants) > 1:
            main_phrase = variants[:-1]
            a.append(variants[-1])

            skills = []

            for i in a:
                skills.append(" ".join(main_phrase + [i]))
            return skills
        else:
            return a + variants
    else:
        return [a]

In [13]:
def split_into_skills(text: str) -> List[str]:
    def _split_into_skills(x):
        # Here we try fix split problem
        # If it is capital letter - maybe new skill...
        x = re.sub(r'([А-ЯA-Z])', r'\n\1', text)
        # Не работает при сокращениях
        return re.split(r'[\n\.]', x)

    pre_skills = list(nonempty(_split_into_skills(text)))
    done_skills = []

    for skill in pre_skills:
        skill = skill.replace('/', ' / ').replace("\xa0", " ")
        if len(re.findall("\([а-яА-ЯA-Za-z\-\s]*\)", skill)) > 0:
            skill1 = re.findall(r'\([а-яА-ЯA-Za-z\-\s]*\)', skill)[0]
            skill2 = re.findall(r'[а-яА-ЯA-Za-z\-\s]*\(', skill)[0][:-1]
            done_skills.extend([skill1, skill2])
        elif len(re.findall("([а-яА-ЯA-Za-z\-\s]*([,и\/]|или)\s[а-яА-ЯA-Za-z\-])", skill)) > 0:
            new_skills = split_skill(skill)
            if new_skills:
                done_skills.extend(new_skills)

        else:
            done_skills.append(skill)
    return done_skills

In [14]:
morph = pymorphy2.MorphAnalyzer()
stopwords = nltk.corpus.stopwords.words(
    'russian') + nltk.corpus.stopwords.words('english')
stopwords += [
    'отличный', 'метр', 'наш', 'клиент', 'банка', 'е', 'проект', 'литр', 'ы', 
    'желательный', 'др', 'ч', 'самый', 'чело', 'мочь', 'хороший', 'год', 'чел', 'чело', 'обязательный'
]

In [15]:
def find_skills(text: str) -> Iterable[str]:
    return cut_by_semmarkers(text) \
        .map(split_into_skills) \
        .get_or([])

In [16]:
helper = {}


def remove_numbers(text):
    return re.sub(r'\d+', '', text)


def normalize_skill(skill: str):

    def get_words(text):
        return re.findall(r'\w+', text)

    def normal_form(word):
        if word not in helper:
            helper[word] = morph.parse(word)[0]
        return helper[word].normal_form

    filter_stopwords = filter(lambda word: word not in stopwords)

    def _get_POS(word):
        if word not in helper:
            helper[word] = morph.parse(word)[0]
        return helper[word].tag.POS

    filter_POS = filter(lambda word: _get_POS(word) in ['NOUN', "ADJF", 'INFN'])
    
    parsed = tuple(
        pipe(
            skill,
            lambda x: x.lower(),
            remove_numbers,
            get_words,
        ))
    
    clear_skill = []
    dirty_skill = []
    
    last_stopword = None
    
    for i in parsed:
        word = normal_form(i)
    
        if word in nltk.corpus.stopwords.words('russian'):
            last_stopword = word
        elif _get_POS(word) in ['NOUN', # NOUN
                                "ADJF", # ADJ
                                'INFN', # VERB
                                'VERB', # VERB
                                'ADJS'  # ADJ
                               ] and len(word) > 3 and word not in stopwords:
            if last_stopword and len(dirty_skill) > 0:
                dirty_skill.append(last_stopword)
                last_stopword = None
            clear_skill.append(word)
            if _get_POS(word) in ['NOUN', 'ADJF']:
                dirty_skill.append(i)
        elif word in ['рф', 'пк']:
            dirty_skill.append(word)
        
    if len(clear_skill) > 1 and len(clear_skill) < 8:
        return clear_skill, dirty_skill
    else:
        return []

In [17]:
model = gensim.models.KeyedVectors.load_word2vec_format("/home/profilum/profilum/ruwikiruscorpora_0_300_20.bin.gz", binary=True)

In [18]:

tmp_vector = np.array([0] * 300, dtype=np.float32)
def _word2vec(word):
    for i in ["_NOUN", "_ADJ", "_VERB"]:
        tmp = "{}{}".format(word, i)
        if tmp in model:
            return model[tmp]
        else:
            return tmp_vector
skill_to_vec = lambda x: sum(list(map(_word2vec, x)))

In [19]:
def get_skills(description: str) -> Iterable[str]:
    skills = tuple(find_skills(description))
    return tuple(pipe(
        skills,
        map(normalize_skill),
        filter(len)
    ))

In [20]:
# a = random.choice(list(all_vacancies))
# i = 0
# while len(get_skills(a.text)) == 0:
#     print("\r Skipped %s" % i, end="")
#     a = random.choice(list(all_vacancies))
#     i+=1
# skills = get_skills(a.text)
# [{"vector": skill_to_vec(i[0]), "skill": " ".join(i[0]), "real_skill": " ".join(i[1])} for i in skills]

In [ ]:
def f(x):
    skills = get_skills(x.text)
    answer = [{"vector": skill_to_vec(i[0]), "skill": " ".join(i[0]), "real_skill": " ".join(i[1])} for i in skills]
    return (x.id, answer)

In [ ]:
values = Parallel(n_jobs=-1, verbose=3, max_nbytes='1G')(delayed(f)(x) for x in all_vacancies)

In [ ]:
vac_to_skills = {c[0]: c[1] for c in tqdm(values)}

In [ ]:
#pickle.dump(vac_to_skills, open("vac_to_skills.pck", "wb"))
vac_to_skills = pickle.load(open("vac_to_skills.pck", "rb"))

In [ ]:
skill_to_id = {}
skill_id_to_skill = {}
skill_to_skill = {}
skill_id_to_vec = {}
r = Counter()
a = 0
j = 0
for key in tqdm(vac_to_skills):
    for _dict in vac_to_skills[key]:
        if _dict["skill"] not in skill_to_id:
            skill_to_id[_dict["skill"]] = a
            skill_id_to_skill[a] = _dict["skill"]
            skill_to_skill[_dict["skill"]] = _dict["real_skill"]
            skill_id_to_vec[a] = _dict["vector"]
            a += 1
        r[_dict["skill"]] += 1

In [9]:
whitelist = set([i[0] for i in r.most_common(500000)])

In [60]:
skill_to_id = {}
skill_id_to_skill = {}
skill_to_skill = {}
skill_id_to_vec = {}
a = 0
j = 0
for key in tqdm(vac_to_skills):
    for _dict in vac_to_skills[key]:
        if _dict["skill"] in whitelist:
            if _dict["skill"] not in skill_to_id:
                skill_to_id[_dict["skill"]] = a
                skill_id_to_skill[a] = _dict["skill"]
                skill_to_skill[_dict["skill"]] = _dict["real_skill"]
                skill_id_to_vec[a] = _dict["vector"]
                a += 1

100%|██████████| 841447/841447 [00:05<00:00, 156544.83it/s]


# Merge

In [ ]:
import theano

In [ ]:
def compile_cos_sim_theano():
    v1 = theano.tensor.vector(dtype='float32')
    v2 = theano.tensor.vector(dtype='float32')
    
    numerator = theano.tensor.sum(v1*v2)
    denominator = theano.tensor.sqrt(theano.tensor.sum(v1**2)*theano.tensor.sum(v2**2))
   
    return theano.function([v1, v2], numerator/denominator)

cos_sim_theano_fn = compile_cos_sim_theano()

In [ ]:
from scipy.sparse import *

In [ ]:
words_to_skills_id = defaultdict(lambda: [])

for _id in skill_id_to_skill:
    for word in set(skill_id_to_skill[_id].split()):
        words_to_skills_id[word].append(_id)

In [ ]:
word_to_id = {}
id_to_word = {}

a = 0
for word in words_to_skills_id.keys():
    word_to_id[word] = a
    id_to_word[a] = word
    a+=1

In [ ]:
mat = dok_matrix((len(words_to_skills_id.keys()), len(skill_id_to_skill.keys())), dtype=np.float16)

for word, skills in tqdm(words_to_skills_id.items()):
    word_id = word_to_id[word]
    for skill in skills:
        mat[word_id, skill] = 1
    
mat = mat.transpose().tocsr()

In [ ]:
# Вспомогательная штука
_skill_id_to_skill = {x: set(skill_id_to_skill[x].split()) for x in skill_id_to_skill}

In [ ]:
# Пересечения скиллов
_len1 = mat * mat.transpose()

In [ ]:
__data = _len1.data
__indptr = _len1.indptr
__indices = _len1.indices

In [ ]:
# Посчитаем длинну
get_lenghts_of_sum_words = lambda current, new: len(_skill_id_to_skill[new]) + len(_skill_id_to_skill[current])

In [11]:
_skill_id_to_skill = {x: set(skill_id_to_skill[x].split()) for x in skill_id_to_skill}
skill_id_to_vec = {x: np.asarray(skill_id_to_vec[x], dtype=np.float32) for x in skill_id_to_vec}

In [12]:
skill_id_to_vec_lambda = lambda x: np.asarray(skill_id_to_vec[x], dtype=np.float32)

In [ ]:
def f(index_current):
    index_new = __indices[__indptr[index_current]:__indptr[index_current+1]]
    lenght = np.asarray([get_lenghts_of_sum_words(index_current, item) for item in index_new])
    iskl = lenght - 2 * __data[__indptr[index_current]:__indptr[index_current+1]]
    
    # Достанем вектора
    sim = map(lambda x: skill_id_to_vec_lambda(x), index_new)
    current_skill_vector = skill_id_to_vec_lambda(index_current)

    # Посчитаем косинус там где нужно
    sim = np.asarray(list(map(lambda x: float(cos_sim_theano_fn(current_skill_vector, x)), sim)))
    sim = sim / iskl
    _len1 = __data[__indptr[index_current]:__indptr[index_current+1]]
    counter = Counter({x:y/0.4 for x,y in zip(index_new, _len1/lenght * sim/iskl)}).most_common()
    answer = []
    
    for i in counter:
        if i[1] > 0.5:
            answer.append(i)
        else:
            break
    
    return (index_current, answer)

In [ ]:
data = Parallel(n_jobs=-1, verbose=3, max_nbytes='10G')(delayed(f)(x) for x in range(1, _len1.shape[0]))

In [ ]:
id_to_simmilar = {x[0]: x[1] for x in tqdm(data)}

In [ ]:
#pickle.dump(id_to_simmilar, open("similars.pck", "wb"))

In [13]:
id_to_simmilar = pickle.load(open("similars.pck", "rb"))

In [14]:
blacklist = set()

for i in tqdm(id_to_simmilar):
    if i not in blacklist:
        a = [k[0] for k in id_to_simmilar[i]]
        if i in a:
            a.remove(i)
        blacklist.update(a)

100%|██████████| 499999/499999 [00:00<00:00, 906642.63it/s]


In [15]:
for i in blacklist:
    if i != 0:
        del id_to_simmilar[i]

# Сервисы
# 1

In [52]:
goal = {}
years = set()

for vac in tqdm(all_vacancies):
    for skill in vac_to_skills[vac.id]:
        if skill['skill'] in goal:
            goal[skill['skill']] += [[
                vac.salary, vac.open_time.year, vac.open_time.month
            ]]
        else:
            goal[skill['skill']] = [[
                vac.salary, vac.open_time.year, vac.open_time.month
            ]]
    years.add(vac.open_time.year)

100%|██████████| 841447/841447 [00:33<00:00, 24830.31it/s]


In [54]:
new_cool = {}
for uskill in tqdm(id_to_simmilar):
    top_similar = [skill_id_to_skill[i[0]] for i in id_to_simmilar[uskill]] + [skill_id_to_skill[uskill]]
    new_cool[skill_to_skill[skill_id_to_skill[uskill]]] = defaultdict(lambda: defaultdict(lambda: 0))

    for skill in top_similar:
        vacs = goal[skill]
        
        for vac in vacs:
            new_cool[skill_to_skill[skill_id_to_skill[uskill]]][vac[1]][vac[2]] += 1

100%|██████████| 341322/341322 [01:24<00:00, 4030.84it/s]


In [ ]:
text = list(filter(lambda x: type(x) is str, list(new_cool.keys())))

In [ ]:
data = pd.DataFrame(text, columns=['text'])

In [ ]:
!rm -rf output/*

In [ ]:
for year in tqdm(list(reversed(list(years)))):
    data_proto = data.copy()

    for month in range(1, 13):
        data_proto['size_%s' % month] = [new_cool[p][year][month]
                                         for p in data_proto['text']]
        #data_proto['size_%s' % month] = data_proto['size_%s' % month]/data_proto['size_%s' % month].max()
        data_proto = data_proto.fillna(0)

    def f(g):
        return sum([g[i] for i in g])

    data_proto['size_year'] = [f(new_cool[p][year])
                               for p in data_proto['text']]
    data_proto = data_proto.sort_values(['size_year'], ascending=False)[
        :20000].to_csv('output/data_{}.csv'.format(year))

In [ ]:
!rm output.zip
!zip -r output.zip output/

# 2

In [16]:
clear_skill_to_vec = {skill_to_skill[x]: skill_id_to_vec[skill_to_id[x]] for x in skill_to_skill}
clear_skill_to_id = {skill_to_skill[x]: skill_to_id[x] for x in skill_to_skill}

In [17]:
id_to_simmilar[2]

[(2, inf), (77280, 0.88587385416030895), (676, 0.87538653612136841)]

In [36]:
from lib import hh_uskills
vectors = []

for main_skill_id in tqdm(id_to_simmilar.keys()):
    skills = [pair[0] for pair in id_to_simmilar[main_skill_id]] + [main_skill_id]
    vector = sum(list(map(lambda _id: skill_id_to_vec[_id], skills))) 
    vectors.append(vector)

items = dict(zip(id_to_simmilar.keys(), vectors))

100%|██████████| 341322/341322 [00:02<00:00, 159748.61it/s]


In [19]:
from MulticoreTSNE import MulticoreTSNE

In [20]:
tsne = MulticoreTSNE(n_jobs=12)

In [43]:
all_vectors = []

for key in list(items.keys()):
    all_vectors.append(items[key])

In [44]:
all_vectors = np.asarray(all_vectors).astype(np.float64)

In [ ]:
all_vectors = tsne.fit_transform(all_vectors)

In [67]:
vectors = {}
for key, i in zip(items.keys(), range(len(all_vectors))):
    vectors[skill_to_skill[skill_id_to_skill[key]]] = all_vectors[i]

In [68]:
c = list(map(lambda x: [x, *vectors[x]], vectors.keys()))

In [69]:
data = pd.DataFrame(c, columns=['text', 'x', 'y'])

In [70]:
def f(g):
    return sum([g[i] for i in g])

In [71]:
for year in tqdm(list(reversed(list(years)))):
    data_proto = data.copy()

    for month in range(1, 13):
        data_proto['size_%s' % month] = [new_cool[p][year][month]
                                         for p in data_proto['text']]
        #data_proto['size_%s' % month] = data_proto['size_%s' % month]/data_proto['size_%s' % month].max()
        data_proto = data_proto.fillna(0)

    def f(g):
        return sum([g[i] for i in g])

    data_proto['size_year'] = [f(new_cool[p][year])
                               for p in data_proto['text']]
    data_proto = data_proto.sort_values(['size_year'], ascending=False)[
        :20000].to_csv('output2/data_{}.csv'.format(year))

  0%|          | 0/16 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [73]:
!mkdir output_csv_uskills

mkdir: cannot create directory ‘output_csv_uskills’: File exists


In [74]:
for year in tqdm(years):
    data_proto = data.copy()

    data_proto['size'] = [f(new_cool[p][year]) for p in data_proto['text']]
    data_proto['size'] = data_proto['size'] / data_proto['size'].max()
    data_proto.sort_values(['size'], ascending=False)[:20000].to_csv(
        'output_csv_uskills/data_%s.csv' % year)

100%|██████████| 16/16 [00:18<00:00,  1.15s/it]


In [ ]:
!rm output_csv_uskills.zip
!zip -r output_csv_uskills.zip output_csv_uskills/

In [83]:
skill_id_to_uskill_id = {}

for key in id_to_simmilar:
    skill_id_to_uskill_id[key] = key
    
    for skill_key in id_to_simmilar[key]:
        skill_id_to_uskill_id[skill_key[0]] = key

In [89]:
len(list(skill_id_to_uskill_id.keys()))

496951

In [85]:
skill_id_to_uskill_id[clear_skill_to_id[vac_to_skills[10000][0]['real_skill']]]

166164

In [92]:
def TLD(v: Vacancy):
    tmp = []
    for i in vac_to_skills[v.id]:
        if i['real_skill'] in clear_skill_to_id:
            my_id = clear_skill_to_id[i['real_skill']]
            if my_id in skill_id_to_uskill_id:
                answer = skill_id_to_uskill_id[my_id]
                if answer not in tmp:
                    tmp.append(skill_to_skill[skill_id_to_skill[answer]])
    return gensim.models.doc2vec.TaggedDocument(tmp, (v.id,))

In [93]:
tlds = tuple(map(TLD, all_vacancies))

In [94]:
tlds = list(filter(None, tlds))

In [95]:
len(tlds)

841447

In [96]:
model = gensim.models.Doc2Vec(
    tlds,
    size=50, window=2, min_count=1, sample=1e-5, iter=10, workers=12)

In [113]:
skill_to_usd = {}
for vac in tqdm(all_vacancies):
    for skill in vac_to_skills[vac.id]:
        skill_name = skill['real_skill']
        if vac.salary != 0:
            if skill_name in skill_to_usd:
                skill_to_usd[skill_name].append(vac.salary)
            else:
                skill_to_usd[skill_name] = [vac.salary]

100%|██████████| 841447/841447 [00:16<00:00, 51306.43it/s]


In [114]:
def mean(arr):
    return np.asarray(arr).mean()

In [115]:
for key in tqdm(skill_to_usd):
    skill_to_usd[key] = mean(skill_to_usd[key])

100%|██████████| 2358899/2358899 [00:20<00:00, 116894.25it/s]


In [116]:
kk = list(model.wv.vocab)

In [ ]:
from flask import Flask, request, render_template, send_from_directory
import os
os.putenv('LANG', 'ru_RU.UTF-8')
os.putenv('LC_ALL', 'ru_RU.UTF-8')
from datetime import timedelta
from flask import make_response, request, current_app
from functools import update_wrapper


def crossdomain(origin=None, methods=None, headers=None,
                max_age=21600, attach_to_all=True,
                automatic_options=True):
    if methods is not None:
        methods = ', '.join(sorted(x.upper() for x in methods))
    if headers is not None:
        headers = ', '.join(x.upper() for x in headers)
    if isinstance(max_age, timedelta):
        max_age = max_age.total_seconds()

    def get_methods():
        if methods is not None:
            return methods

        options_resp = current_app.make_default_options_response()
        return options_resp.headers['allow']

    def decorator(f):
        def wrapped_function(*args, **kwargs):
            if automatic_options and request.method == 'OPTIONS':
                resp = current_app.make_default_options_response()
            else:
                resp = make_response(f(*args, **kwargs))
            if not attach_to_all and request.method != 'OPTIONS':
                return resp

            h = resp.headers

            h['Access-Control-Allow-Origin'] = origin
            h['Access-Control-Allow-Methods'] = get_methods()
            h['Access-Control-Max-Age'] = str(max_age)
            if headers is not None:
                h['Access-Control-Allow-Headers'] = headers
            return resp

        f.provide_automatic_options = False
        return update_wrapper(wrapped_function, f)
    return decorator


app = Flask(__name__)


@app.route("/get_all")
@crossdomain(origin='*')
def k():
    now = request.args['term']

    tmp = []
    for i in kk:
        if now in i:
            tmp.append(i)

            if len(tmp) == 10:
                break

    return json.dumps(tmp)


@app.route("/demo")
@crossdomain(origin='*')
def j():
    return render_template('demo.html')


@app.route("/demo2")
@crossdomain(origin='*')
def jj():
    return render_template('demo2.html')


@app.route('/output_csv/<path:path>')
def send_js(path):
    return send_from_directory('static/js/output_csv', path)


@app.route("/get_word")
@crossdomain(origin='*')
def hello():
    ans = list(filter(lambda x: len(x) > 0, eval(request.args['words'])))
    ans = model.most_similar_cosmul(positive=ans)
    ans = sorted([[mean(skill_to_usd[i[0]]), i[0]] for i in ans])
    ans = list(reversed(ans))
    return json.dumps({'skills': ans})


if __name__ == "__main__":
    app.run(host="88.99.252.225", port=8081)

 * Running on http://88.99.252.225:8081/ (Press CTRL+C to quit)
94.127.216.116 - - [12/Jan/2018 11:48:44] "GET /demo HTTP/1.1" 200 -
94.127.216.116 - - [12/Jan/2018 11:48:46] "GET /get_all?term=c HTTP/1.1" 200 -
94.127.216.116 - - [12/Jan/2018 11:48:52] "GET /get_word?words=%5B%22c%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D0%BE%D0%B5+%D0%BC%D1%8B%D1%88%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%22%2C%22%22%5D HTTP/1.1" 200 -
94.127.216.116 - - [12/Jan/2018 11:48:59] "GET /demo2 HTTP/1.1" 200 -
94.127.216.116 - - [12/Jan/2018 11:48:59] "GET /static/css/scatter.css HTTP/1.1" 200 -
94.127.216.116 - - [12/Jan/2018 11:48:59] "GET /static/js/scatter.js HTTP/1.1" 200 -
94.127.216.116 - - [12/Jan/2018 11:48:59] "GET /static/js/d3-tip.js HTTP/1.1" 200 -
94.127.216.116 - - [12/Jan/2018 11:48:59] "GET /static/images/tick.png HTTP/1.1" 200 -
94.127.216.116 - - [12/Jan/2018 11:49:00] "GET /output_csv/data_2003.csv HTTP/1.1" 200 -
94.127.216.116 - - [12/Jan/2018 11:50:19] "GET /output_csv/data_2004.csv HTTP/1.1" 200 